In [1]:
!pip install --upgrade --quiet google-genai

In [2]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [3]:
import os

PROJECT_ID = "genai-458310"  # @param {type: "string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [4]:
from IPython.display import Markdown, display
from google.genai.types import (
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    Tool,
    VertexAISearch,
)

In [5]:
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}

In [6]:
PROMPT = "When is the next solar eclipse in the US?"

In [7]:
def print_grounding_data(response: GenerateContentResponse) -> None:
    """Prints Gemini response with grounding citations in Markdown format."""
    if not (response.candidates and response.candidates[0].grounding_metadata):
        print("Response does not contain grounding metadata.")
        display(Markdown(response.text))
        return

    grounding_metadata = response.candidates[0].grounding_metadata
    markdown_parts = []

    # Citation indexes are in bytes
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)
    last_byte_index = 0

    for support in grounding_metadata.grounding_supports:
        markdown_parts.append(
            text_bytes[last_byte_index : support.segment.end_index].decode(ENCODING)
        )

        # Generate and append citation footnotes (e.g., "[1][2]")
        footnotes = "".join([f"[{i + 1}]" for i in support.grounding_chunk_indices])
        markdown_parts.append(f" {footnotes}")

        # Update index for the next segment
        last_byte_index = support.segment.end_index

    # Append any remaining text after the last citation
    if last_byte_index < len(text_bytes):
        markdown_parts.append(text_bytes[last_byte_index:].decode(ENCODING))

    markdown_parts.append("\n\n----\n## Grounding Sources\n")

    # Build Grounding Sources Section
    markdown_parts.append("### Grounding Chunks\n")
    for i, chunk in enumerate(grounding_metadata.grounding_chunks, start=1):
        context = chunk.web or chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk without context: {chunk}")
            continue

        uri = context.uri
        title = context.title or "Source"

        # Convert GCS URIs to public HTTPS URLs
        if uri and uri.startswith("gs://"):
            uri = uri.replace("gs://", "https://storage.googleapis.com/", 1).replace(
                " ", "%20"
            )
        markdown_parts.append(f"{i}. [{title}]({uri})\n")

    # Add Search/Retrieval Queries
    if grounding_metadata.web_search_queries:
        markdown_parts.append(
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_parts.append(
                f"\n**Search Entry Point:**\n{grounding_metadata.search_entry_point.rendered_content}\n"
            )
    elif grounding_metadata.retrieval_queries:
        markdown_parts.append(
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    display(Markdown("".join(markdown_parts)))

In [8]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
)

display(Markdown(response.text))

The next solar eclipse in the U.S. will be an **annular solar eclipse on October 14, 2023**.

After that, the next **total solar eclipse** visible in the U.S. will be on **April 8, 2024**.


In [9]:
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[google_search_tool]),
)

print_grounding_data(response)

The next solar eclipse visible in the contiguous United States will be a total solar eclipse on August 23, 2044. [1][2][3] However, if you include Alaska, there will be a total solar eclipse on March 30, 2033. [3][1]


----
## Grounding Sources
### Grounding Chunks
1. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIC3EPljRlFIKaRP2P7Abow1G30k3Y_40PVkv9gV0CN5MS21mGsx6UscTISmRj_CKbFVIutmDI3hMIFyckiQDzqDXKASsBNP4IlY4ksWN5kGY-0j_ykFNrPjMtNkyLJtbXKYMUUyET4Ta_QjZ3rCl3xx1EAWCatyA==)
2. [cbsnews.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAK0SwBsrABLu0Rc2FKZ2jcGzueNCaJ6hwZ7nBr975Dqh2BEjs38L74r740X_j62eTj_YtF4NarCixtOqXu5rW77ZkQY7FW6uu7pqYKptfvIvKNbpiamlU2jSMhqsxUBPHRsGSDQr-0wv-mDngPfTRFCqJLBiihfZszhE74=)
3. [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALffhKc0SQnnOG75sAsbOIqHQR2Si4uTScuw3jImPlkBIqKlKOp84norwegeUCHaKZFiyRgfhlSN6h5jl7_pIxwlRJg86oY-myq04XrzF4ez9wj42-nL7c5tDUk1eR585jMxsVFlOlaw54MVPxGYErgwCIv8uoed7l4SrLxX1c3qFJVRhz8RZo8Lh0D1nh4)

**Web Search Queries:** ['next solar eclipse in the US']

**Search Entry Point:**
<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJsihsDc452uXISj80RxyQayY6-Tj9HELSIB6ijVOLEzKYvPjnc71uAQHp5FKtjz3Q5u3RVVTRVsU2WxWXJEWyzXgmvsiVUsLn0TbXZVx6ceeFjo4fhzbInDb-uv9OWiIFeB-uIsFhz3h4IkPZoCfkTxq46O4TzTcqHKikc3oM3X2ywcyiJe3-iFfeZBZaeOrFIKldfVDNnYdP3qoPljDM=">next solar eclipse in the US</a>
  </div>
</div>



In [10]:
PROMPT = "What is the current temperature at this location?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri="gs://github-repo/generative-ai/gemini/grounding/paris.jpg",
            mime_type="image/jpeg",
        ),
        PROMPT,
    ],
    config=GenerateContentConfig(
        tools=[google_search_tool],
    ),
)

print_grounding_data(response)

The image shows a view of Paris, France, featuring the Eiffel Tower and the Seine River. However, I cannot determine the current temperature at this location based on the image alone. To provide you with the current temperature, I would need to access a weather service. Would you like me to search for the current temperature in Paris?
The current temperature in Paris, France is 77°F (25°C). [1] It feels like 77°F (25°C). [2][1] The chance of rain is around 0% and the humidity is around 27%. [2] It is sunny. [3][4][2][5][6][1]


----
## Grounding Sources
### Grounding Chunks
1. [yahoo.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIsHtyIlFs3NW8UhPv685Ku-D1YLh1lak1gxmdUHmTYjri3Ajqi8V08SX-RBG8dZxrAhPyt_ZZPTwOqHvjJQpqcNfTM_B5e0r3oVhfqiMsg42rjsWCvBlXblMU0sbfC4nQ5Bb0XKua1yDDy3pd-I0xpmzAakSKSXADv4W8MRjtLDTBz)
2. [Weather information for locality: Paris](https://www.google.com/search?q=weather+in+Paris)
3. [timeanddate.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI7mKsXsfb0TPYs-suSZNdrr_ct_qLKwJ7sZ1xIPNeJTha7m__J5mv_L15J20hzF6LqgZF_LbeXSwxlMrssAFAHZb2zdM6vj_r6OYyrhBB77tsAYkx1jSgQzL7m9egTEhp441Kv27MgpN7NPqv9eA==)
4. [wunderground.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJYKoBwcnjkhG2YkBFbGRSzbLIHf8DyboItggX3DsXUMRgZCZLHxIrS61xvXiH6jTx-RLaVd7mvvfLhVrTWiPMZSDmhiT8P3xI10RPjE0ObglBpnWkALmMmxp2EtrxjXlz5ZVVyM0RC)
5. [weatherapi.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKIiiVChL6NhVPJRcsbBfxRp2_W7irjVhLQnqZ-j7DNvoFfy-boeoPJxn8HgsweCKzcu0EuquH4Mq9BuiFpePMA_BWB9qJnd47BAYczeMQD3ErPVerULDfiAQdcfDKnx3lk--54qrrQMLrmXQ==)
6. [metoffice.gov.uk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALlf6hHTSJFQM72IfxhM_tgRKBVrHEdYBWdxptx4vKgGA_EUjujWeesiJCVgNa8XuTt5Q0Z_N0S1sUnKJuWHvYuZ8hG6l40EDP917G3Jj3ZIiPAgKTaKf0-UhBr7chuGmO_jIgF3FALDA0eauZY)

**Web Search Queries:** ['current temperature in Paris']

**Search Entry Point:**
<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKUT8un776AcUmioILzhVvdAoA7G6CIcOvG99LY4UuLUAgMaYVWmcL9k_EbN2FvSUgFz5Br6AnHkU4EcjQtK8rxJ5ktdckkCpAX73jBVJzUTuWPGAVAUfY3Ff0AUypX_KplRjHowmR5jVO7oYzSJdXeyFUzy5BnE_Ef2x4u4wK8k7Gktm70xyCB_x_R6lht1Xgpu51T6pCAg5yQwNPY8w==">current temperature in Paris</a>
  </div>
</div>



In [12]:
%%bash -s "$PROJECT_ID" "$MODEL_ID"
PROJECT_ID=$1
MODEL_ID=$2
curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "x-server-timeout: 60" \
  https://us-central1-aiplatform.googleapis.com/v1/projects/\$PROJECT_ID/locations/us-central1/publishers/google/models/\$MODEL_ID:generateContent \
  -d '{
    "contents": [{
      "role": "user",
      "parts": [{
        "text": "Who won the 2024 UEFA European Championship?"
      }]
    }],
    "tools": [{
      "enterpriseWebSearch": {}
    }]
  }'

{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "Spain won the 2024 UEFA European Championship, securing their record-breaking fourth title by defeating England 2-1 in the final. The match was held at the Olympiastadion in Berlin on July 14, 2024.\n"
          }
        ]
      },
      "finishReason": "STOP",
      "groundingMetadata": {
        "webSearchQueries": [
          "who won UEFA European Championship 2024"
        ],
        "searchEntryPoint": {
          "renderedContent": "\u003cstyle\u003e\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: solid 1px;\n  border-radius: 16px;\n  min-width: 14px;\n  padding: 5px 16px;\n  text-align: center;\n  user-select: none;\n  margin: 0 8px;\n  -webkit-tap-highlight-color: tr

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8659    0  8460  100   199   2945     69  0:00:02  0:00:02 --:--:--  3014
